In [232]:
import numpy as np
import pandas as pd

In [233]:
# instructions
# rename the updated server file to 'report'
# rename Azimi's report to 'azimi'
# rename the current sheet in Azimi's reprt to 'reprot'
# run the main file.
# open the final_report excel file.
# copy and paste the content to the 'گزارش سفاله' file
# print as PDF

In [234]:
customer_codes = pd.read_excel('Excel_report/azimi.xlsx', sheet_name='کد تفصیلی', index_col=0)
Azimi_report = pd.read_excel('Excel_report/azimi.xlsx', sheet_name='report', index_col=0)
receipt = pd.read_excel('Excel_report/report.xlsx', sheet_name='Receipt', index_col=0)
delivery = pd.read_excel('Excel_report/report.xlsx', sheet_name='Delivery', index_col=0)

collective = pd.concat([receipt,delivery], axis=0)
collective = collective.sort_values('تاريخ میلادی')
collective = collective.reset_index()


In [235]:
first_day_of_the_year = ['2022-03-21','2023-03-21','2024-03-20']
not_in_azimi = []
date_filter = first_day_of_the_year + not_in_azimi
collective = collective.loc[~collective['تاريخ میلادی'].isin(date_filter),:].copy()
# collective[(collective['تاريخ میلادی'].dt.year== 2024) & (collective['تاريخ میلادی'].dt.month == 3)]

In [236]:
collective['بدهکار'] = -collective.loc[collective['مقدار (کیلوگرم)']<0,'مقدار (کیلوگرم)']
collective['بستانکار'] = collective.loc[collective['مقدار (کیلوگرم)']>0,'مقدار (کیلوگرم)']
collective = collective.fillna(0)

In [237]:
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
collective['بدهکار تجمیعی'] = group_by_cust['بدهکار'].cumsum()
collective['بستانکار تجمیعی'] = group_by_cust['بستانکار'].cumsum()

collective['مانده در خط'] = collective['بدهکار تجمیعی'] - collective['بستانکار تجمیعی']

In [238]:
def not_settled_inline(collective):
    collective['مانده درخط تسویه نشده'] = collective['بدهکار تجمیعی'] - collective['بستانکار'].sum()
    collective.loc[collective['مانده درخط تسویه نشده']<0,'مانده درخط تسویه نشده'] = 0
    return collective
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
collective = group_by_cust.apply(not_settled_inline)

In [239]:
collective['سفارش تسویه شده'] = collective['بدهکار'] - collective['مانده درخط تسویه نشده']
collective.loc[collective['سفارش تسویه شده']<0,'سفارش تسویه شده'] = 0
collective['سفارش تسویه نشده'] = collective['بدهکار'] - collective['سفارش تسویه شده']

In [240]:
# set end date
report_end_date = pd.Timestamp.now()
# report_end_date = pd.to_datetime('2024-06-29')

collective = collective[collective['تاريخ میلادی']<= report_end_date]

credit_in_days = 15

In [241]:
collective['روز از ارسال سفارش تسویه نشده'] = pd.Timedelta(days=0)
collective.loc[collective['سفارش تسویه نشده']>0,'روز از ارسال سفارش تسویه نشده'] = report_end_date - collective['تاريخ میلادی']
collective['روز از ارسال سفارش تسویه نشده'] = collective['روز از ارسال سفارش تسویه نشده'].dt.days

collective['تسویه نشده × روز از ارسال'] = collective['سفارش تسویه نشده'] * collective['روز از ارسال سفارش تسویه نشده']

In [242]:
collective['تاخیر از سررسید'] = 0
collective.loc[collective['سفارش تسویه نشده']>0,'تاخیر از سررسید'] = collective['روز از ارسال سفارش تسویه نشده'] - credit_in_days
collective.loc[collective['تاخیر از سررسید']<0,'تاخیر از سررسید'] = 0

collective['سفارش سررسید شده'] = collective.loc[collective['تاخیر از سررسید']>0,'سفارش تسویه نشده']
collective['سفارش سررسید شده'] = collective['سفارش سررسید شده'].fillna(0)

collective['سررسید شده × تاخیر از سررسید'] = collective['سفارش سررسید شده'] * collective['تاخیر از سررسید']

In [243]:
collective['سررسید شده × روز از ارسال'] = collective['سفارش سررسید شده'] * collective['روز از ارسال سفارش تسویه نشده']

In [244]:
columns = pd.Series(collective.columns)
# columns

In [245]:
excel_file_path = 'Excel_report/collective.xlsx'
# collective.to_excel(excel_file_path, index=False)

# filter collective by customer
collective[collective['کد ردیابی']==30068]

,تاريخ,تاريخ میلادی,کد ردیابی,مقدار (کیلوگرم),بدهکار,بستانکار,بدهکار تجمیعی,بستانکار تجمیعی,مانده در خط,مانده درخط تسویه نشده,سفارش تسویه شده,سفارش تسویه نشده,روز از ارسال سفارش تسویه نشده,تسویه نشده × روز از ارسال,تاخیر از سررسید,سفارش سررسید شده,سررسید شده × تاخیر از سررسید,سررسید شده × روز از ارسال
3757,01/02/19,2022-05-09,30068,-4710.0,4710.0,0.0,4710.0,0.0,4710.0,0.0,4710.0,0.0,0,0.0,0,0.0,0.0,0.0
3806,01/02/25,2022-05-15,30068,4834.0,0.0,4834.0,4710.0,4834.0,-124.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0
3831,01/02/28,2022-05-18,30068,2266.0,0.0,2266.0,4710.0,7100.0,-2390.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0
3868,01/02/31,2022-05-21,30068,-9511.0,9511.0,0.0,14221.0,7100.0,7121.0,0.0,9511.0,0.0,0,0.0,0,0.0,0.0,0.0
3911,01/03/03,2022-05-24,30068,5936.0,0.0,5936.0,14221.0,13036.0,1185.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10265,03/03/05,2024-05-25,30068,-8103.0,8103.0,0.0,119574.0,122468.0,-2894.0,0.0,8103.0,0.0,0,0.0,0,0.0,0.0,0.0
10269,03/03/05,2024-05-25,30068,-3015.0,3015.0,0.0,122589.0,122468.0,121.0,0.0,3015.0,0.0,0,0.0,0,0.0,0.0,0.0
10512,03/04/10,2024-06-30,30068,1132.0,0.0,1132.0,122589.0,123600.0,-1011.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0
10513,03/04/10,2024-06-30,30068,454.0,0.0,454.0,122589.0,124054.0,-1465.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0


In [246]:
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
group_by_cust_sum = group_by_cust.sum(numeric_only=True)
group_by_cust_sum.loc[30068]
group_by_cust_sum['مانده سفاله'] = (group_by_cust_sum['بدهکار'] - group_by_cust_sum['بستانکار']).round(0).fillna(0).astype(int)
group_by_cust_sum['متوسط زمان تسویه نشده'] = (group_by_cust_sum['تسویه نشده × روز از ارسال'] / group_by_cust_sum['سفارش تسویه نشده']).round(0).fillna(0).astype(int)
group_by_cust_sum['متوسط زمان سررسید شده'] = (group_by_cust_sum['سررسید شده × روز از ارسال'] / group_by_cust_sum['سفارش سررسید شده']).round(0).fillna(0).astype(int)
group_by_cust_sum['متوسط تاخیر از سررسید'] = (group_by_cust_sum['سررسید شده × تاخیر از سررسید'] / group_by_cust_sum['سفارش سررسید شده']).round(0).fillna(0).astype(int)

In [247]:
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)

group_by_cust_max_days = group_by_cust.agg({'روز از ارسال سفارش تسویه نشده': 'max'})
group_by_cust_max_days = group_by_cust.agg({'روز از ارسال سفارش تسویه نشده': 'max'}).rename(columns={'روز از ارسال سفارش تسویه نشده': 'بازه تسویه نشده'})
merg_sum_days = pd.merge(group_by_cust_sum,group_by_cust_max_days,on='کد ردیابی')

In [248]:
columns = pd.Series(merg_sum_days.columns)
columns

0                   مقدار (کیلوگرم)
1                            بدهکار
2                          بستانکار
3                     بدهکار تجمیعی
4                   بستانکار تجمیعی
5                       مانده در خط
6             مانده درخط تسویه نشده
7                   سفارش تسویه شده
8                  سفارش تسویه نشده
9     روز از ارسال سفارش تسویه نشده
10        تسویه نشده × روز از ارسال
11                  تاخیر از سررسید
12                 سفارش سررسید شده
13     سررسید شده × تاخیر از سررسید
14        سررسید شده × روز از ارسال
15                      مانده سفاله
16            متوسط زمان تسویه نشده
17            متوسط زمان سررسید شده
18            متوسط تاخیر از سررسید
19                  بازه تسویه نشده
dtype: object

In [249]:
report = merg_sum_days.iloc[:,[15,8,19,16,12,17,18]]

In [250]:
report

,مانده سفاله,سفارش تسویه نشده,بازه تسویه نشده,متوسط زمان تسویه نشده,سفارش سررسید شده,متوسط زمان سررسید شده,متوسط تاخیر از سررسید
کد ردیابی,,,,,,,
10002,-689,0.0,0,0,0.0,0,0
10003,21540,21540.0,56,30,15539.0,36,21
10004,26,26.0,124,124,26.0,124,109
10007,52,52.5,1198,1198,52.5,1198,1183
10008,578,578.0,1198,1198,578.0,1198,1183
...,...,...,...,...,...,...,...
50006,7581,7581.0,1198,1198,7581.0,1198,1183
50011,-8,0.0,0,0,0.0,0,0
50016,-2632,0.0,0,0,0.0,0,0


In [251]:
Azimi_report = Azimi_report.iloc[:-1,:]
Azimi_report = Azimi_report.loc[Azimi_report['مانده سوفاله']<0,:]
Azimi_report = Azimi_report.rename(columns={'مانده سوفاله':'مانده سفاله عظیمی'})
Azimi_report.index = Azimi_report.index.astype(int)
report_filterred_by_azimi = pd.merge(Azimi_report[['عنوان','مانده سفاله عظیمی']],report,left_index=True,right_index=True,how='left')
report_filterred_by_azimi['مغایرت عظیمی'] = report_filterred_by_azimi['مانده سفاله عظیمی'] + report_filterred_by_azimi['مانده سفاله']
report_filterred_by_azimi['سفارش تسویه نشده'] = report_filterred_by_azimi['سفارش تسویه نشده'].fillna(0)
report_filterred_by_azimi['سفارش سررسید شده'] = report_filterred_by_azimi['سفارش سررسید شده'].fillna(0)
report_filterred_by_azimi = report_filterred_by_azimi.fillna('-')
report_filterred_by_azimi

,عنوان,مانده سفاله عظیمی,مانده سفاله,سفارش تسویه نشده,بازه تسویه نشده,متوسط زمان تسویه نشده,سفارش سررسید شده,متوسط زمان سررسید شده,متوسط تاخیر از سررسید,مغایرت عظیمی
كد تفصيلي,,,,,,,,,,
30004,توليدي صنعتي صفاشير پارس,-11837.0,11837,11837.0,9,6,0.0,0,0,0.0
30068,شيرگاز ايران,-121.0,-5248,0.0,0,0,0.0,0,0,-5369.0
30127,شير گاز آذران,-17759.0,17759,17759.0,21,13,2998.0,21,6,0.0
10003,اصغر نورعلي,-21540.0,21540,21540.0,56,30,15539.0,36,21,0.0
30162,شركت الماس آريا,-132.0,132,132.0,68,68,132.0,68,53,0.0
30009,شركت ايمن تك پيشرو,-28957.0,28957,28957.0,34,15,10517.0,29,14,0.0
10055,مريم فيض آبادي,-3450.0,3450,3450.0,21,5,524.0,21,6,0.0
10027,مصطفي نورعلي,-7645.0,7645,7645.0,16,9,556.0,16,1,0.0
10015,مصدقي,-4042.0,4042,4042.0,12,10,0.0,0,0,0.0


In [252]:
final_report = report_filterred_by_azimi.iloc[:,[0,2,4,5,6,7,8]]
final_report

,عنوان,مانده سفاله,بازه تسویه نشده,متوسط زمان تسویه نشده,سفارش سررسید شده,متوسط زمان سررسید شده,متوسط تاخیر از سررسید
كد تفصيلي,,,,,,,
30004,توليدي صنعتي صفاشير پارس,11837,9,6,0.0,0,0
30068,شيرگاز ايران,-5248,0,0,0.0,0,0
30127,شير گاز آذران,17759,21,13,2998.0,21,6
10003,اصغر نورعلي,21540,56,30,15539.0,36,21
30162,شركت الماس آريا,132,68,68,132.0,68,53
30009,شركت ايمن تك پيشرو,28957,34,15,10517.0,29,14
10055,مريم فيض آبادي,3450,21,5,524.0,21,6
10027,مصطفي نورعلي,7645,16,9,556.0,16,1
10015,مصدقي,4042,12,10,0.0,0,0


In [254]:
excel_file_path = 'Excel_report/final_report.xlsx'
final_report.to_excel(excel_file_path, index=False)

In [ ]:
# extras to be implemented later
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
collective['تعداد کل روزهای بدهکار'] = group_by_cust['تاريخ میلادی'].diff()
group_by_cust = collective.groupby('کد ردیابی')
collective['تعداد کل روزهای بدهکار'] = group_by_cust['تعداد کل روزهای بدهکار'].shift(-1)

def assign_to_last(group, column):
    value = report_end_date - group.iloc[-1,group.columns.get_loc('تاريخ میلادی')]
    group.iloc[-1, group.columns.get_loc(column)] = value
    return group

group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
collective = group_by_cust.apply(assign_to_last,column = 'تعداد کل روزهای بدهکار')

collective['مانده در خط تسویه شده'] = collective['مانده در خط'] - collective['مانده درخط تسویه نشده']
